## TF-IDF Nedir?

**TF-IDF**, bir kelimenin bir dokümanda ne kadar önemli olduğunu ölçen bir vektörleştirme yöntemidir.  
- **TF (Term Frequency):** Bir kelimenin dokümanda kaç kez geçtiği.
- **IDF (Inverse Document Frequency):** O kelimenin tüm dokümanlar arasında ne kadar nadir olduğu.

TF-IDF, sık geçen ama her dokümanda olan kelimelere düşük, nadir ve ayırt edici kelimelere yüksek değer verir.

Örnek Metinler

In [1]:
documents = [
    "Bugün hava çok güzel ve güneşli.",
    "Yarın hava yağmurlu olacak.",
    "Bugün parka gittik ve çok eğlendik.",
    "Yarın da parka gitmek istiyoruz."
]

1. scikit-learn ile TF-IDF

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(lowercase=True)
X = vectorizer.fit_transform(documents)

print("Kelimeler (Vocab):", vectorizer.get_feature_names_out())
print("TF-IDF Vektörleri:\n", X.toarray())

Kelimeler (Vocab): ['bugün' 'da' 'eğlendik' 'gitmek' 'gittik' 'güneşli' 'güzel' 'hava'
 'istiyoruz' 'olacak' 'parka' 've' 'yarın' 'yağmurlu' 'çok']
TF-IDF Vektörleri:
 [[0.37222485 0.         0.         0.         0.         0.47212003
  0.47212003 0.37222485 0.         0.         0.         0.37222485
  0.         0.         0.37222485]
 [0.         0.         0.         0.         0.         0.
  0.         0.43779123 0.         0.55528266 0.         0.
  0.43779123 0.55528266 0.        ]
 [0.37222485 0.         0.47212003 0.         0.47212003 0.
  0.         0.         0.         0.         0.37222485 0.37222485
  0.         0.         0.37222485]
 [0.         0.48546061 0.         0.48546061 0.         0.
  0.         0.         0.48546061 0.         0.38274272 0.
  0.38274272 0.         0.        ]]


2. Stopwords ile TF-IDF
* Türkçe stopwords’leri kaldırarak daha anlamlı TF-IDF vektörleri elde edebilirsin:

In [3]:
turkish_stopwords = [
    've', 'bir', 'bu', 'şu', 'o', 'ben', 'sen', 'biz', 'siz', 'onlar',
    'için', 'ile', 'da', 'de', 'ta', 'te', 'den', 'dan', 'ten', 'tan',
    'nin', 'nın', 'nun', 'nün', 'in', 'ın', 'un', 'ün',
    'ya', 'ye', 'na', 'ne', 'a', 'e', 'ı', 'i', 'u', 'ü', 'o', 'ö',
    'çok', 'daha', 'en', 'hem', 'hiç', 'her', 'hangi', 'kendi',
    'sonra', 'önce', 'şimdi', 'bugün', 'yarın', 'ama', 'fakat', 'ancak', 'lakin', 'ki', 'eğer', 'ise',
    'gibi', 'kadar', 'göre', 'doğru', 'karşı', 'rağmen',
    'mi', 'mı', 'mu', 'mü', 'midir', 'mıdır', 'mudur', 'müdür'
]

vectorizer = TfidfVectorizer(lowercase=True, stop_words=turkish_stopwords)
X = vectorizer.fit_transform(documents)

print("Stopwords çıkarıldıktan sonra kelimeler:", vectorizer.get_feature_names_out())
print("TF-IDF Vektörleri:\n", X.toarray())

Stopwords çıkarıldıktan sonra kelimeler: ['eğlendik' 'gitmek' 'gittik' 'güneşli' 'güzel' 'hava' 'istiyoruz'
 'olacak' 'parka' 'yağmurlu']
TF-IDF Vektörleri:
 [[0.         0.         0.         0.61761437 0.61761437 0.48693426
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.48693426
  0.         0.61761437 0.         0.61761437]
 [0.61761437 0.         0.61761437 0.         0.         0.
  0.         0.         0.48693426 0.        ]
 [0.         0.61761437 0.         0.         0.         0.
  0.61761437 0.         0.48693426 0.        ]]


3. Lemmatize Edilmiş TF-IDF (Zeyrek veya TRNLP ile)
* Önce her dokümandaki kelimeleri lemmatize edip, sonra TF-IDF uygulayabilirsin.
* Aşağıda Zeyrek ile örnek var (TRNLP için yukarıdaki BoW örneğindeki gibi lemmatize fonksiyonunu kullanabilirsin):

In [4]:
import re

try:
    import zeyrek
    analyzer = zeyrek.MorphAnalyzer()
    zeyrek_available = True
except ImportError:
    print("Zeyrek kurulu değil.")
    zeyrek_available = False

def lemmatize_with_zeyrek(text):
    words = re.findall(r'\w+', text.lower(), re.UNICODE)
    lemmas = []
    for word in words:
        try:
            analyses = analyzer.analyze(word)
            if analyses and len(analyses) > 0 and len(analyses[0]) > 1 and analyses[0][1]:
                lemma = analyses[0][1][0]
            else:
                lemma = word
            # Boş string veya None ise ekleme
            if lemma and lemma.strip():
                lemmas.append(lemma)
        except Exception:
            lemmas.append(word)
    return lemmas

if zeyrek_available:
    lemmatized_docs = [' '.join(lemmatize_with_zeyrek(doc)) for doc in documents]
    # Boş dokümanları at
    lemmatized_docs = [doc for doc in lemmatized_docs if doc.strip()]
    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(lemmatized_docs)
    print("Zeyrek ile lemmatize edilmiş kelimeler (her doküman):")
    for i, doc in enumerate(lemmatized_docs):
        print(f"Doc {i+1}: {doc}")
    print("TF-IDF Vektörleri (Zeyrek lemmatize):\n", X.toarray())
    print("Kelimeler (Vocab):", vectorizer.get_feature_names_out())
else:
    print("Zeyrek kurulu değil veya import edilemedi.")

APPENDING RESULT: <(bugün_Adv)(-)(bugün:advRoot_ST)>
APPENDING RESULT: <(bugün_Noun_Time)(-)(bugün:noun_S + a3sg_S + pnon_S + nom_ST)>
APPENDING RESULT: <(hava_Adj)(-)(hava:adjectiveRoot_ST)>
APPENDING RESULT: <(hav_Noun)(-)(hav:noun_S + a3sg_S + pnon_S + a:dat_ST)>
APPENDING RESULT: <(hava_Noun)(-)(hava:noun_S + a3sg_S + pnon_S + nom_ST)>
APPENDING RESULT: <(çok_Postp_PCAbl)(-)(çok:postpRoot_ST)>
APPENDING RESULT: <(çok_Adj)(-)(çok:adjectiveRoot_ST)>
APPENDING RESULT: <(çok_Det)(-)(çok:detRoot_ST)>
APPENDING RESULT: <(çok_Adv)(-)(çok:advRoot_ST)>
APPENDING RESULT: <(güzel_Adj)(-)(güzel:adjectiveRoot_ST)>
APPENDING RESULT: <(güzel_Adv)(-)(güzel:advRoot_ST)>
APPENDING RESULT: <(güzel_Noun)(-)(güzel:noun_S + a3sg_S + pnon_S + nom_ST)>
APPENDING RESULT: <(ve_Conj)(-)(ve:conjRoot_ST)>
APPENDING RESULT: <(güneş_Noun)(-)(güneş:noun_S + a3sg_S + pnon_S + nom_ST + li:with_S + adjectiveRoot_ST)>
APPENDING RESULT: <(Güneş_Noun_Prop)(-)(güneş:nounProper_S + a3sg_S + pnon_S + nom_ST + li:with_S + 

Zeyrek ile lemmatize edilmiş kelimeler (her doküman):
Doc 1: bugün hava çok güzel ve güneşli
Doc 2: yarın hava yağmurlu olacak
Doc 3: bugün parka gittik ve çok eğlendik
Doc 4: yarın da parka gitmek istiyoruz
TF-IDF Vektörleri (Zeyrek lemmatize):
 [[0.37222485 0.         0.         0.         0.         0.47212003
  0.47212003 0.37222485 0.         0.         0.         0.37222485
  0.         0.         0.37222485]
 [0.         0.         0.         0.         0.         0.
  0.         0.43779123 0.         0.55528266 0.         0.
  0.43779123 0.55528266 0.        ]
 [0.37222485 0.         0.47212003 0.         0.47212003 0.
  0.         0.         0.         0.         0.37222485 0.37222485
  0.         0.         0.37222485]
 [0.         0.48546061 0.         0.48546061 0.         0.
  0.         0.         0.48546061 0.         0.38274272 0.
  0.38274272 0.         0.        ]]
Kelimeler (Vocab): ['bugün' 'da' 'eğlendik' 'gitmek' 'gittik' 'güneşli' 'güzel' 'hava'
 'istiyoruz' 'ola

4. NLTK ile TF-IDF
* NLTK’da doğrudan TF-IDF yoktur, ama kelimeleri tokenize edip scikit-learn ile kullanabilirsin:

In [5]:
import nltk

tokenized_docs = [' '.join(nltk.word_tokenize(doc.lower())) for doc in documents]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(tokenized_docs)
print("NLTK ile tokenize edilmiş kelimeler:", vectorizer.get_feature_names_out())
print("TF-IDF Vektörleri:\n", X.toarray())

NLTK ile tokenize edilmiş kelimeler: ['bugün' 'da' 'eğlendik' 'gitmek' 'gittik' 'güneşli' 'güzel' 'hava'
 'istiyoruz' 'olacak' 'parka' 've' 'yarın' 'yağmurlu' 'çok']
TF-IDF Vektörleri:
 [[0.37222485 0.         0.         0.         0.         0.47212003
  0.47212003 0.37222485 0.         0.         0.         0.37222485
  0.         0.         0.37222485]
 [0.         0.         0.         0.         0.         0.
  0.         0.43779123 0.         0.55528266 0.         0.
  0.43779123 0.55528266 0.        ]
 [0.37222485 0.         0.47212003 0.         0.47212003 0.
  0.         0.         0.         0.         0.37222485 0.37222485
  0.         0.         0.37222485]
 [0.         0.48546061 0.         0.48546061 0.         0.
  0.         0.         0.48546061 0.         0.38274272 0.
  0.38274272 0.         0.        ]]


5. spaCy ile TF-IDF
* spaCy ile kelimeleri tokenize edip, scikit-learn ile TF-IDF hesaplayabilirsin:

In [6]:
import spacy

nlp = spacy.blank("tr")
tokenized_docs = []
for doc in documents:
    tokens = [token.text.lower() for token in nlp(doc) if token.is_alpha]
    tokenized_docs.append(' '.join(tokens))

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(tokenized_docs)
print("spaCy ile tokenize edilmiş kelimeler:", vectorizer.get_feature_names_out())
print("TF-IDF Vektörleri:\n", X.toarray())

spaCy ile tokenize edilmiş kelimeler: ['bugün' 'da' 'eğlendik' 'gitmek' 'gittik' 'güneşli' 'güzel' 'hava'
 'istiyoruz' 'olacak' 'parka' 've' 'yarın' 'yağmurlu' 'çok']
TF-IDF Vektörleri:
 [[0.37222485 0.         0.         0.         0.         0.47212003
  0.47212003 0.37222485 0.         0.         0.         0.37222485
  0.         0.         0.37222485]
 [0.         0.         0.         0.         0.         0.
  0.         0.43779123 0.         0.55528266 0.         0.
  0.43779123 0.55528266 0.        ]
 [0.37222485 0.         0.47212003 0.         0.47212003 0.
  0.         0.         0.         0.         0.37222485 0.37222485
  0.         0.         0.37222485]
 [0.         0.48546061 0.         0.48546061 0.         0.
  0.         0.         0.48546061 0.         0.38274272 0.
  0.38274272 0.         0.        ]]


## Pratik Notlar

- **TF-IDF**, metin sınıflandırma, bilgi çıkarımı ve arama motorlarında çok kullanılır.
- Türkçe’de lemmatizasyon ve stopwords temizliği, TF-IDF’in kalitesini artırır.
- scikit-learn, hızlı ve pratik bir çözümdür.